### The Basics of LangChain

In this notebook we'll explore exactly what LangChain is doing - and implement a straightforward example that lets us ask questions of any document we want!

First things first, let's get our dependencies all set!

In [ ]:
!pip install openai langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.4 MB/s eta 0:00:00


You'll need to have an OpenAI API key for this next part - see [this](https://www.onmsft.com/how-to/how-to-get-an-openai-api-key/) if you haven't already set one up!

In [ ]:
import os
import openai
#add your openai api key
openai.api_key = ""

#### Helper Functions (run this cell)

In [ ]:
from IPython.display import display, Markdown

def disp_markdown(text: str) -> None:
  display(Markdown(text))

### Our First LangChain ChatModel



---


<div class="warn">Note: Information on OpenAI's <a href=https://openai.com/pricing>pricing</a> and <a href=https://openai.com/policies/usage-policies>usage policies.</a></div>



---



Now that we're set-up with OpenAI's API - we can begin making our first ChatModel!

There's a few important things to consider when we're using LangChain's ChatModel that are outlined [here](https://python.langchain.com/en/latest/modules/models/chat.html)

Let's begin by initializing the model with OpenAI's `gpt-3.5-turbo` (ChatGPT) model.

We're not going to be leveraging the [streaming](https://python.langchain.com/en/latest/modules/models/chat/examples/streaming.html) capabilities in this Notebook - just the basics to get us started!

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai.api_key)

If we look at the [Chat completions](https://platform.openai.com/docs/guides/chat) documentation for OpenAI's chat models - we'll see that there are a few specific fields we'll need to concern ourselves with:

`role`
- This refers to one of three "roles" that interact with the model in specific ways.
- The `system` role is an optional role that can be used to guide the model toward a specific task. Examples of `system` messages might be:
  - You are an expert in Python, please answer questions as though we were in a peer coding session.
  - You are the world's leading expert in stamps.

  These messages help us "prime" the model to be more aligned with our desired task!

- The `user` role represents, well, the user!
- The `assistant` role lets us act in the place of the model's outputs. We can (and will) leverage this for some few-shot prompt engineering!

Each of these roles has a class in LangChain to make it nice and easy for us to use!

Let's look at an example.

In [ ]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# The SystemMessage is associated with the system role
system_message = SystemMessage(content="You are a food critic.")

# The HumanMessage is associated with the user role
user_message = HumanMessage(content="Do you think Kraft Dinner constitues fine dining?")

# The AIMessage is associated with the assistant role
assistant_message = AIMessage(content="Egads! No, it most certainly does not!")

Now that we have those messages set-up, let's send them to `gpt-3.5-turbo` with a new user message and see how it does!

It's easy enough to do this - the ChatOpenAI model accepts a list of inputs!

In [ ]:
second_user_message = HumanMessage(content="What about Red Lobster, surely that is fine dining!")

# create the list of prompts
list_of_prompts = [
    system_message,
    user_message,
    assistant_message,
    second_user_message
]

# we can just call our chat_model on the list of prompts!
chat_model(list_of_prompts)

AIMessage(content='While Red Lobster is a popular seafood chain restaurant, it is not traditionally considered "fine dining." Fine dining establishments focus on providing a high-end dining experience with exceptional service, attention to detail, and often use high-quality ingredients. Red Lobster, while delicious in its own right, is more casual and family-friendly in its approach.', additional_kwargs={}, example=False)

Great! That's inline with what we expected to see!

### PromptTemplates

Next stop, we'll discuss a few templates. This allows us to easily interact with our model by not having to redo work we've already completed!

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# we can signify variables we want access to by wrapping them in {}
system_prompt_template = "You are an expert in {SUBJECT}, and you're currently feeling {MOOD}"
system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt_template)

user_prompt_template = "{CONTENT}"
user_prompt_template = HumanMessagePromptTemplate.from_template(user_prompt_template)

# put them together into a ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, user_prompt_template])

Now that we have our `chat_prompt` set-up with the templates - let's see how we can easily format them with our content!

NOTE: `disp_markdown` is just a helper function to display the formatted markdown response.

In [ ]:
# note the method `to_messages()`, that's what converts our formatted prompt into
formatted_chat_prompt = chat_prompt.format_prompt(SUBJECT="cheeses", MOOD="quite tired", CONTENT="Hi, what are the finest cheeses?").to_messages()

disp_markdown(chat_model(formatted_chat_prompt).content)

Hello! As an expert in cheeses, there are many types of fine cheeses that I could recommend. However, here are a few that are widely considered to be among the finest:

1. Parmigiano-Reggiano: This hard Italian cheese is aged for at least two years and has a sharp, nutty flavor that pairs well with many foods.

2. Roquefort: This French blue cheese is made from sheep's milk and has a sharp, tangy flavor with a creamy texture.

3. Gouda: This Dutch cheese comes in many varieties, but the aged Gouda is especially prized for its nutty, caramel-like flavor.

4. Brie de Meaux: This soft, creamy French cheese has a buttery flavor and a bloomy rind that adds depth to its flavor.

5. Comté: This French cheese is made from raw cow's milk and has a nutty, sweet flavor with a slightly fruity finish.

Of course, there are many other fine cheeses out there, and the best one for you will depend on your personal taste preferences.

### Putting the Chain in LangChain

In essense, a chain is exactly as it sounds - it helps us chain actions together.

Let's take a look at an example.

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_model, prompt=chat_prompt)

disp_markdown(chain.run(SUBJECT="classic cars", MOOD="angry", CONTENT="Is the 67 Chevrolet Impala a good vehicle?"))

As an AI language model, I do not have feelings, so I cannot feel angry. However, regarding your question, the 1967 Chevrolet Impala is widely considered to be an excellent classic car. It has a sleek and stylish design, and its powerful V8 engine delivers impressive performance. It was also featured in popular TV shows such as Supernatural, which has helped to increase its popularity among car enthusiasts. Overall, the '67 Impala is a highly sought-after vehicle in the classic car community and is considered a great investment for collectors.

### Incorporate A Local Document

Now that we've got our first chain running, let's talk about how we can leverage our own document!

First off, we'll need a document!

For this example, we'll be using Devil wears Prada script.

In [ ]:
#!wget https://erki.lap.ee/failid/raamatud/guide1.txt

In [ ]:
! cd /content/scripts

In [ ]:
with open("/content/scripts/devil_wears_prada.txt") as f:
    devil_wears_prada = f.read()

Next we'll want to split our text into appropirately sized chunks.

We're going to be using the [CharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/character_text_splitter.html) from LangChain today.

The size of these chunks will depend heavily on a number of factors relating to which LLM you're using, what the max context size is, and more.

You can also choose to have the chunks overlap to avoid potentially missing any important information between chunks. As we're dealing with a novel - there's not a critical need to include overlap.

We can also pass in the separator - this is what we'll try and separate the documents on. Be careful to understand your documents so you can be sure you use a valid separator!

For now, we'll go with 1000 characters.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator = "\n")
texts = text_splitter.split_text(devil_wears_prada)

In [ ]:
len(texts)

247

Now that we've split our document into more manageable sized chunks. We'll need to embed those documents!

For more information on embedding - please check out [this](https://platform.openai.com/docs/guides/embeddings) resource from OpenAI.

In order to do this, we'll first need to select a method to embed - for this example we'll be using OpenAI's embedding - but you're free to use whatever you'd like.

You just need to ensure you're using consistent embeddings as they don't play well with others.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = openai.api_key

embeddings = OpenAIEmbeddings()

Now that we've set up how we want to embed our document - we'll need to embed it.

For this week we'll be glossing over the technical details of this process - as we'll get more into next week.

Just know that we're converting our text into an easily queryable format!

We're going to leverage ChromaDB for this example, so we'll want to install that dependency.

In [ ]:
!pip install chromadb tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.7/922.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.7 MB/s e

In [ ]:
from langchain.vectorstores import Chroma

docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

Now that we have our documents embedded we're free to query them with natural language! Let's see this in action!

In [ ]:
query = "What makes towels important?"
docs = docsearch.get_relevant_documents(query)

In [ ]:
docs[0]

Document(page_content='intercut ANDY getting ready and...\n\n       ... we see three or four other GIRLS getting ready too.\n\n1A     A drawer filled with about forty lipsticks slides into                1A\n       frame... One of the GIRLS carefully applies shiny lip gloss\n       with a brush... ANDY puts on cherry Chapstick...\n\n1B&C   A lacy thong floats through the air... one of the GIRLS pulls        1B&C\n       it up her glossy legs... A comfy cotton pair of Jockey\n       bikinis is tugged out of a pile... ANDY pulls them on...\n\n1D     A gorgeous pair of slingback heels. One of the GIRLS pulls            1D\n       the shoes on while her model handsome boyfriend reclines on\n       crisp white sheets, watching her.\n\n1E     A pair of comfortable wedges... ANDY dresses while NATE,              1E\n       rumpled, unshowered, wearing an old Alice in Chains t-shirt,\n       watches her, reclining in mismatched bed-in-a-bag sheets...', metadata={'source': '1'})

Finally, we're able to combine what we've done so far into a chain!

We're going to leverage the `load_qa_chain` to quickly integrate our queryable documents with an LLM.

There are 4 major methods of building this chain, they can be found [here](https://docs.langchain.com/docs/components/chains/index_related_chains)!

For this example we'll be using the `stuff` chain type.

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
#query = "How much cheese is in this book?" # answer - i dont know
query = "Who is Andy in this script?"
docs = docsearch.get_relevant_documents(query)
chain.run(input_documents=docs, question=query)

" Andy is a recent college graduate who is trying to get a job as Miranda Priestly's assistant."